# Lecture 14 : Causality

## 6.4: Module 6 Notebook 4 ##

In [ ]:
from datascience import *
import numpy as np

%matplotlib inline
import matplotlib.pyplot as plots
plots.style.use('fivethirtyeight')

## Randomized Control Experiment

In [ ]:
botox = Table.read_table('bta.csv')
botox.show()

In [ ]:
botox.pivot('Result', 'Group')

In [ ]:
botox.group('Group', np.average)

## Hypotheses: All **Potential** Scores

In [ ]:
# what do we mean by **all potential** scores for the control/treatment categories?

# this table, while representing the same data as botox, is clearly including all potential observations
observed_outcomes = Table.read_table("observed_outcomes.csv")
observed_outcomes.show()

# Testing the Hypothesis

In [ ]:
# we created this function in the previous notebook (6.3, let's reuse it)
def difference_of_means(table, label, group_label):
    """Takes: name of table, column label of numerical variable,
    column label of group-label variable
    Returns: Difference of means of the two groups"""
    
    #table with the two relevant columns
    reduced = table.select(label, group_label)  
    
    # table containing group means
    means_table = reduced.group(group_label, np.average)
    # array of group means
    means = means_table.column(1)
    
    return means.item(1) - means.item(0)

In [ ]:
observed_diff = difference_of_means(botox, 'Result', 'Group')
observed_diff

In [ ]:
# we created this too in the previous notebook (6.3, let's reuse it)
def one_simulated_difference(table, label, group_label):
    """Takes: name of table, column label of numerical variable,
    column label of group-label variable
    Returns: Difference of means of the two groups after shuffling labels"""
    
    # array of shuffled labels
    shuffled_labels = table.sample(with_replacement = False
                                                    ).column(group_label)
    
    # table of numerical variable and shuffled labels
    shuffled_table = table.select(label).with_column(
        'Shuffled Label', shuffled_labels)
    
    return difference_of_means(shuffled_table, label, 'Shuffled Label')

In [ ]:
one_simulated_difference(botox, 'Result', 'Group')

In [ ]:
simulated_diffs = make_array()

for i in np.arange(10000):
    sim_diff = one_simulated_difference(botox, 'Result', 'Group')
    simulated_diffs = np.append(simulated_diffs, sim_diff)

In [ ]:
col_name = 'Distances between groups'
Table().with_column(col_name, simulated_diffs).hist(col_name)

In [ ]:
col_name = 'Distances between groups'
Table().with_column(col_name, simulated_diffs).hist(col_name)
plots.scatter(observed_diff, -0.001, color='red', s=30);

In [ ]:
# p-value
sum(simulated_diffs >= observed_diff)/len(simulated_diffs)

In [ ]:
# the test shows that the data is more consistent (i.e., better supports) the alternative hypothesis